In [3]:
import pandas as pd

college = pd.read_csv('new_college.csv')
dictionary = pd.read_csv('CollegeScorecard_Raw_Data/CollegeScorecardDataDictionary-09-12-2015.csv')

In [1]:
#print college

In [6]:
allcollegelist = []
for i in range(len(college)):
    allcollegelist.append(i)

In [6]:
alg_categories = ['CONTROL', 
                  'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'SATWR25', 'SATWR75',
                  'NPT4_PUB', 'NPT4_PRIV']

alg_race = ['UGDS','UGDS_WHITE', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN',
            'UGDS_AIAN', 'UGDS_NHPI', 'UGDS_2MOR', 'UGDS_NRA', 'UGDS_UNKN',
            'UGDS_WHITENH', 'UGDS_API', 'UGDS_API']

alg_majors = ['PCIP01', 'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11',
              'PCIP12', 'PCIP13', 'PCIP14', 'PCIP15', 
              'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24', 'PCIP25', 'PCIP26', 'PCIP27',
              'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38', 'PCIP39', 'PCIP40', 'PCIP41', 'PCIP42',
              'PCIP43', 'PCIP44', 'PCIP45', 'PCIP46',
              'PCIP47', 'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51', 'PCIP52', 'PCIP54']

disp_categories = ['OPEID','INSTNM','CITY','STABBR', 'INSTURL', 'HIGHDEG','ADM_RATE']

In [7]:
"""inputs = [sat_reading, sat_writing, sat_math, pref_major, race_diversity, public_private, popul, pay]
sat_reading = Input SAT Reading Score (integer)
sat_writing = Input SAT Writing Score (integer)
sat_math = Input SAT Math Score (Integer)
pref_major = Type in string for desired major
race_diversity = Input [0,1,2] For Low, Medium, High
public_private = Input [0,1] for Public, Private
popul = Input [0,1,2] For Low, Medium, High
pay = Input [0,1,2] For Low, Medium, High
"""



'inputs = [sat_reading, sat_writing, sat_math, pref_major, race_diversity, public_private, popul, pay]\nsat_reading = Input SAT Reading Score (integer)\nsat_writing = Input SAT Writing Score (integer)\nsat_math = Input SAT Math Score (Integer)\npref_major = Type in string for desired major\nrace_diversity = Input [0,1,2] For Low, Medium, High\npublic_private = Input [0,1] for Public, Private\npopul = Input [0,1,2] For Low, Medium, High\npay = Input [0,1,2] For Low, Medium, High\n'

In [69]:
#Creating lists with id of private college and public college
privatecollege = []
publiccollege = []
for i in allcollegelist:
    if str(college['NPT4_PUB'][i]) != 'nan':
        publiccollege.append(i)
    if str(college['NPT4_PRIV'][i]) != 'nan':
        privatecollege.append(i)
        
print '# private colleges:', len(privatecollege)
print '# public colleges:', len(publiccollege)
print '# all colleges:', len(allcollegelist)

# private colleges: 4753
# public colleges: 1923
# all colleges: 7804


In [5]:
#creating a dictionary where the key is the major variable and the value is the label (name of the major)
dmajor = {}
for i in range(len(dictionary)):
    if str(dictionary.label[i]) != 'nan':
        dmajor[dictionary['VARIABLE NAME'][i]] = dictionary.label[i]
print dmajor

{'PCIP04': 'Architecture and Related Services', 'PCIP05': 'Area, Ethnic, Cultural, Gender, and Group Studies', 'PCIP24': 'Liberal Arts and Sciences, General Studies and Humanities', 'PCIP25': 'Library Science', 'PCIP26': 'Biological and Biomedical Sciences', 'PCIP01': 'Agriculture, Agriculture Operations, and Related Sciences', 'PCIP42': 'Psychology', 'PCIP29': 'Military Technologies and Applied Sciences', 'PCIP40': 'Physical Sciences', 'PCIP41': 'Science Technologies/Technicians', 'PCIP46': 'Construction Trades', 'PCIP22': 'Legal Professions and Studies', 'PCIP44': 'Public Administration and Social Service Professions', 'PCIP09': 'Communication, Journalism, and Related Programs', 'PCIP23': 'English Language and Literature/Letters', 'PCIP47': 'Mechanic and Repair Technologies/Technicians', 'PCIP03': 'Natural Resources and Conservation', 'PCIP39': 'Theology and Religious Vocations', 'PCIP27': 'Mathematics and Statistics', 'PCIP43': 'Homeland Security, Law Enforcement, Firefighting and R

In [46]:
def each_major(searchstr, collegelist):
    filteredlist = []
    for key in dmajor:
        if searchstr.lower() in dmajor[key].lower():
            for i in collegelist:
                if college[key][i] > 0.05:
                    if i not in filteredlist:
                        filteredlist.append(i)
                    
    return filteredlist

def major(search, collegelist):
    searchlst = search.split(';')
    filteredlist = []
    for searchstr in searchlst:
        searchlist = each_major(searchstr, collegelist)
        for i in searchlist:
            if i not in filteredlist:
                filteredlist.append(i)
    return filteredlist

In [47]:
def each_sat(score, column, collegelist, percentile):
    filteredlist = []
    
    if percentile == 25:
        for i in collegelist:
            if score >= college[column][i]-50:
                filteredlist.append(i)
    
    if percentile == 75:
        for i in collegelist:
            if score <= college[column][i]+50:
                filteredlist.append(i)
        
    return filteredlist

def section_sat(score, section, collegelist):
    """
    section = 'VR' or 'WR' or 'MT'
    """
    column25 = 'SAT'+section+'25'
    column75 = 'SAT'+section+'75'
    
    list25 = each_sat(score, column25, collegelist, 25)
    list75 = each_sat(score, column75, list25, 75)
    
    return list75

def sat(vrscore, wrscore, mtscore, collegelist):
    """
    vrscore: reading score (int)
    wrscore: writing score (int)
    mtscore: math score (int)
    collegelist (list)
    """
    listvr = section_sat(vrscore, 'VR', collegelist)
    listwr = section_sat(wrscore, 'WR', listvr)
    listmt = section_sat(mtscore, 'MT', listwr)
    
    return listmt

In [63]:
collegelist = sat(800,800,800,allcollegelist)
search = 'engINEering; literature'
collegelist = major(search, collegelist)

for i in collegelist:
        print college['INSTNM'][i]

California Institute of Technology
Harvey Mudd College
Northwestern University
Massachusetts Institute of Technology
Tufts University
Washington University in St Louis
Dartmouth College
Princeton University
Columbia University in the City of New York
Duke University
University of Pennsylvania
Swarthmore College
Brown University
Vanderbilt University
Rice University
Stanford University
Franklin W Olin College of Engineering
Yale University
University of Chicago
Bowdoin College
Amherst College
Wellesley College
Williams College
Carleton College
